# XSAR example

open a dataset with [xsar.open_dataset](../basic_api.rst#xsar.open_dataset)

In [ ]:
import xsar
import os
import numpy as np

In [ ]:
# get test file. You can replace with an path to other SAFE
filename = xsar.get_test_file('S1A_IW_GRDH_1SDV_20170907T103020_20170907T103045_018268_01EB76_Z010.SAFE')

## Open a dataset with a xsar.SentinelMeta object
A [xsar.SentinelMeta](../basic_api.rst#xsar.SentinelMeta) object handles all attributes and methods that can't be embdeded in a `xarray.Dataset` object.
It can also replace a filename in [xsar.open_dataset](../basic_api.rst#xsar.open_dataset)

When using it in a notebook, it also have an html representation.

In [ ]:
sar_meta = xsar.SentinelMeta(filename)
sar_meta

A [xsar.SentinelMeta](../basic_api.rst#xsar.SentinelMeta) object handles all attributes and methods relative to the SAFE metadata that can't be embdeded in a `xarray.Dataset` object.

It can be used to construct a [xsar.SentinelDataset](../basic_api.rst#xsar.SentinelDataset) object.

In [ ]:
sar_ds_obj = xsar.SentinelDataset(sar_meta)
sar_ds_obj

a [xsar.SentinelDataset](../basic_api.rst#xsar.SentinelDataset) as a `.dataset` property that is the already known `xarray.Dataset` returned by [xsar.open_dataset](../basic_api.rst#xsar.open_dataset)

So, those commands are equivalent:
```python
ds = xsar.open_dataset(filename)
ds = xsar.SentinelDataset(xsar.SentinelMeta(filename)).dataset
```

By using the underlying objects, the user have access to properties and methods that are not available to the `xarray.Dataset` object.

## Open a dataset at lower resolution

We can use [xsar.SentinelMeta](../basic_api.rst#xsar.SentinelMeta) properties to compute the pixel count we need to average to get a resolution of 200*200 meters

In [ ]:
resolution = {'atrack' : int(np.round(200 / sar_meta.pixel_atrack_m)), 'xtrack': int(np.round(200 / sar_meta.pixel_xtrack_m))}
resolution

In [ ]:
sar_ds_obj = xsar.SentinelDataset(sar_meta, resolution = resolution)
{k:sar_ds_obj.dataset.attrs[k] for k in ['coverage','pixel_atrack_m','pixel_xtrack_m'] }

## Extract a sub image of 10*10km around a lon/lat point

### Convert (lon,lat) to (atrack, xtrack)
we can use [xsar.SentinelMeta.ll2coords](../basic_api.rst#xsar.SentinelMeta.ll2coords) to convert (lon,lat) to (atrack, xtrack):

In [ ]:
# from a shapely object
point_lonlat =  sar_ds_obj.s1meta.footprint.centroid
point_coords = sar_ds_obj.s1meta.ll2coords(point_lonlat.x, point_lonlat.y)
point_coords

The result is floating, because it's the position inside the pixel.
To get the real coordinates in an existing dataset, we can use [xsar.SentinelDataset.ll2coords](../basic_api.rst#xsar.SentinelDataset.ll2coords)

In [ ]:
point_coords = sar_ds_obj.ll2coords(point_lonlat.x, point_lonlat.y)
point_coords

### Extract the sub-image

In [ ]:
box_size = 10000 # 10km
dist = {'atrack' : int(np.round(box_size / 2 / sar_meta.pixel_atrack_m)), 'xtrack': int(np.round(box_size / 2 / sar_meta.pixel_xtrack_m))}
dist

In [ ]:
# select 10*10 km around point_coords
sar_ds_obj.dataset = sar_ds_obj.dataset.sel(atrack=slice(point_coords[0] - dist['atrack'], point_coords[0] + dist['atrack']), xtrack=slice(point_coords[1] - dist['xtrack'], point_coords[1] + dist['xtrack']))
sar_ds_obj.dataset